In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import signal

# --- Data Loading and Preprocessing ---

def load_and_preprocess_ecg(file_path):
    df = pd.read_csv('C:/Users/abdulssekyanzi/EDA Dataset.csv/100.csv')
    ecg_signal = df['MLII'].values  # or 'V5'
    return ecg_signal

def preprocess_ecg_signal(ecg_signal):
    # Noise reduction (Butterworth filter)
    b, a = signal.butter(3, [0.5, 40], btype='bandpass', fs=360) #360 is the sample rate of the MIT-BIH dataset
    ecg_filtered = signal.lfilter(b, a, ecg_signal)

    # Standardization
    scaler = StandardScaler()
    ecg_scaled = scaler.fit_transform(ecg_filtered.reshape(-1, 1)).flatten()
    return ecg_scaled

def segment_heartbeats(ecg_signal, rpeaks):
    heartbeats = []
    for i in range(len(rpeaks) - 1):
        start = rpeaks[i] - 100  # Adjust as needed
        end = rpeaks[i + 1] + 100  # Adjust as needed
        if start >= 0 and end < len(ecg_signal):
            segment = ecg_signal[start:end]
            segment_resampled = signal.resample(segment, 250) #Resample to a fixed length
            heartbeats.append(segment_resampled)
    return np.array(heartbeats)



In [2]:
# Example Usage (replace with your data loading and labeling logic):
# Note: You will need to write the code that assigns labels to each segmented heartbeat.
#       This is highly dataset specific.
#       For simplicity, I will create some dummy heartbeats and labels.

#Dummy data creation. Replace with your actual data loading.
ecg_signal = np.random.randn(10000)
ecg_processed = preprocess_ecg_signal(ecg_signal)
rpeaks, _ = signal.find_peaks(ecg_processed, height=0.5)
heartbeats = segment_heartbeats(ecg_processed, rpeaks)
labels = np.random.randint(0, 5, size=len(heartbeats)) # 5 classes for example

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
labels_one_hot = tf.keras.utils.to_categorical(labels, num_classes=num_classes)

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(heartbeats, labels_one_hot, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# --- Teacher Model ---

teacher_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, 5, activation='relu', input_shape=(250, 1)),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

teacher_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
teacher_model.fit(X_train[..., np.newaxis], y_train, epochs=49, validation_data=(X_val[..., np.newaxis], y_val))

# --- Student Model ---

student_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=(250, 1)),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])



C:\Users\abdulssekyanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/49
15/15 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.2139 - loss: 1.6188 - val_accuracy: 0.1942 - val_loss: 1.6090
Epoch 2/49
15/15 ━━━━━━━━━━━━━━━━━━━━ 13s 470ms/step - accuracy: 0.2250 - loss: 1.6073 - val_accuracy: 0.2233 - val_loss: 1.6098
Epoch 3/49
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 447ms/step - accuracy: 0.1759 - loss: 1.6036 - val_accuracy: 0.2913 - val_loss: 1.5975
Epoch 4/49
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 425ms/step - accuracy: 0.2314 - loss: 1.6036 - val_accuracy: 0.2136 - val_loss: 1.5936
Epoch 5/49
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 380ms/step - accuracy: 0.1989 - loss: 1.6026 - val_accuracy: 0.2039 - val_loss: 1.5995
Epoch 6/49
15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 362ms/step - accuracy: 0.2246 - loss: 1.5992 - val_accuracy: 0.2136 - val_loss: 1.5978
Epoch 7/49
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 436ms/step - accuracy: 0.2306 - loss: 1.5969 - val_accuracy: 0.2330 - val_loss: 1.5900
Epoch 8/49
15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 366ms/step - accuracy: 0.2474 - loss: 1.5942 - val_accuracy: 0

In [3]:
# --- Knowledge Distillation ---

temperature = 5.0
alpha = 0.5

def distillation_loss(teacher_logits, student_logits, temperature):
    teacher_probs = tf.nn.softmax(teacher_logits / temperature)
    student_probs = tf.nn.softmax(student_logits / temperature)
    return tf.keras.losses.KLDivergence()(teacher_probs, student_probs)

def combined_loss(y_true, y_pred, teacher_logits, student_logits, temperature, alpha):
    ce_loss = tf.keras.losses.CategoricalCrossentropy()(y_true, y_pred)
    dist_loss = distillation_loss(teacher_logits, student_logits, temperature)
    return alpha * dist_loss + (1 - alpha) * ce_loss



In [4]:
# Distillation Training Loop
optimizer = tf.keras.optimizers.Adam()

for epoch in range(60):
    with tf.GradientTape() as tape:
        teacher_logits = teacher_model(X_train[..., np.newaxis])
        student_logits = student_model(X_train[..., np.newaxis])
        loss = combined_loss(y_train, student_logits, teacher_logits, student_logits, temperature, alpha)

    gradients = tape.gradient(loss, student_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, student_model.trainable_variables))

    if epoch % 1 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Compile the student model before evaluation
student_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# --- Evaluation ---

teacher_loss, teacher_accuracy = teacher_model.evaluate(X_test[..., np.newaxis], y_test)
student_loss, student_accuracy = student_model.evaluate(X_test[..., np.newaxis], y_test)

print(f"Teacher Accuracy: {teacher_accuracy}, Student Accuracy: {student_accuracy}")

Epoch 0, Loss: 0.8071150779724121
Epoch 1, Loss: 0.8059360980987549
Epoch 2, Loss: 0.8049037456512451
Epoch 3, Loss: 0.8040049076080322
Epoch 4, Loss: 0.8032239079475403
Epoch 5, Loss: 0.8025428652763367
Epoch 6, Loss: 0.8019425272941589
Epoch 7, Loss: 0.8014036417007446
Epoch 8, Loss: 0.8009107112884521
Epoch 9, Loss: 0.8004509210586548
Epoch 10, Loss: 0.8000159859657288
Epoch 11, Loss: 0.7995997667312622
Epoch 12, Loss: 0.7992002964019775
Epoch 13, Loss: 0.798818051815033
Epoch 14, Loss: 0.7984519004821777
Epoch 15, Loss: 0.7980989217758179
Epoch 16, Loss: 0.7977550625801086
Epoch 17, Loss: 0.7974151968955994
Epoch 18, Loss: 0.7970750331878662
Epoch 19, Loss: 0.7967350482940674
Epoch 20, Loss: 0.7963979244232178
Epoch 21, Loss: 0.7960624694824219
Epoch 22, Loss: 0.7957274913787842
Epoch 23, Loss: 0.795385479927063
Epoch 24, Loss: 0.7950316667556763
Epoch 25, Loss: 0.7946624159812927
Epoch 26, Loss: 0.794278621673584
Epoch 27, Loss: 0.7938820123672485
Epoch 28, Loss: 0.793469965457916